In [2]:
// SETUP
use std::fs;
use std::fs::File;
use std::collections::{HashMap, HashSet};
use std::str::FromStr;
use std::io::{BufRead, BufReader, Error, ErrorKind, Read};
use std::time::Instant;

fn read_string(path: &str) -> Result<String, Error> {
    let data = fs::read_to_string(path)?;
    Ok(data)
}

fn read_lines(path: &str) -> Result<Vec<String>, Error> {
    let file = File::open(path)?; 
    BufReader::new(file).lines().collect()
}

fn read_integers(path: &str) -> Result<Vec<i64>, Error> {
    let mut v = Vec::new();
    for line in read_lines(path)? {
        let n = line   
            .trim() 
            .parse() 
            .map_err(|e| Error::new(ErrorKind::InvalidData, e))?; 
        v.push(n);
    }
    Ok(v)
}

const DATA_ROOT: &str = "../data/";

fn data_file(problem_id: u32) -> String {
    format!("{}/{:02}.txt", DATA_ROOT, problem_id)
}

fn test_file(problem_id: u32, test_id: u32) -> String {
    format!("{}/{:02}t{}.txt", DATA_ROOT, problem_id, test_id)
}

fn extra_file(problem_id: u32, suffix: &str) -> String {
    format!("{}/{:02}.{}.txt", DATA_ROOT, problem_id, suffix)
}

In [13]:
// DAY01
extern crate kth;
use kth::SliceExtKth;

fn problem01() {    
    let data = read_lines(&data_file(1)).unwrap();
    let cals: Vec<Vec<i64>> = data.into_iter().fold(Vec::new(), |mut res, x| {
        if x == "" || res.is_empty() {
            res.push(Vec::new());
        } else {
            res.last_mut().unwrap().push(x.parse().unwrap());
        }
        res
    });
    let mut sums: Vec<i64> = cals.iter().map(|x| x.iter().sum()).collect();
    let res1: i64 = *sums.iter().max().unwrap();
    let n = sums.len();
    sums.partition_by_kth(n - 3);
    let res2: i64 = sums.iter().rev().take(3).sum();
    println!("Answer 1: {}, Answer 2: {}", res1, res2);
}

problem01();

Answer 1: 71934, Answer 2: 211447


In [75]:
// DAY02
#[derive(Debug, PartialEq, Copy, Clone)]
enum Move {
    Rock,
    Paper,
    Scissors,
}

#[derive(Debug, PartialEq, Copy, Clone)]
enum MoveOutcome {
    Win,
    Draw,
    Loss,
}

impl FromStr for Move {
    type Err = ();
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        match s {
            "A" | "X" => Ok(Move::Rock),
            "B" | "Y" => Ok(Move::Paper),
            "C" | "Z" => Ok(Move::Scissors),
            _ => Err(()),
        }
    }
}

impl FromStr for MoveOutcome {
    type Err = ();
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        match s {
            "X" => Ok(MoveOutcome::Loss),
            "Y" => Ok(MoveOutcome::Draw),
            "Z" => Ok(MoveOutcome::Win),
            _ => Err(()),
        }
    }
}

impl Move {
    fn score(&self) -> i64 {
        match self {
            Self::Rock => 1,
            Self::Paper => 2,
            Self::Scissors => 3, 
        }
    }
}

impl MoveOutcome {
    fn score(&self) -> i64 {
        match self {
            Self::Win => 6,
            Self::Draw => 3,
            Self::Loss => 0, 
        }
    }
}

fn beats(m1: &Move, m2: &Move) -> bool {
    match (m1, m2) {
        (Move::Rock, Move::Scissors) | 
        (Move::Paper, Move::Rock) | 
        (Move::Scissors, Move::Paper) => true,
        _ => false,
    }
}

fn get_outcome(m1: &Move, m2: &Move) -> MoveOutcome {
    if m1 == m2 { return MoveOutcome::Draw; }
    else if beats(m1, m2) { return MoveOutcome::Win; }
    else { return MoveOutcome::Loss; }
}

fn get_move_for_outcome(m: &Move, outcome: &MoveOutcome) -> Move {
    match outcome {
        MoveOutcome::Draw => *m,
        MoveOutcome::Win => match m {
            Move::Rock => Move::Paper,
            Move::Paper => Move::Scissors,
            Move::Scissors => Move::Rock,
        },
        MoveOutcome::Loss => match m {
            Move::Rock => Move::Scissors,
            Move::Paper => Move::Rock,
            Move::Scissors => Move::Paper,        
        },
    }
}

fn problem02() {    
    let data = read_lines(&data_file(2)).unwrap();
    let moves: Vec<Vec<&str>> = data.iter()
        .map(|line| line.split(" ").collect())
        .collect();
    
    let moves1: Vec<(Move, Move)> = moves.iter()
        .map(|s| (s[0].parse().unwrap(), s[1].parse().unwrap()))
        .collect();
    let moves2: Vec<(Move, MoveOutcome)> = moves.iter()
        .map(|s| (s[0].parse().unwrap(), s[1].parse().unwrap()))
        .collect();
    
    let res1: i64 = moves1.iter()
        .map(|(m1, m2)| m2.score() + get_outcome(m2, m1).score())
        .sum();
    
    let res2: i64 = moves2.iter()
        .map(|(m, r)| r.score() + get_move_for_outcome(m, r).score())
        .sum();

    println!("Answer 1: {}, Answer 2: {}", res1, res2);
}

problem02();

Answer 1: 13526, Answer 2: 14204


In [138]:
// DAY03
use std::collections::HashSet;

fn halve(s: &str) -> (&str, &str) {
    let n = s.len() / 2;
    (&s[..n], &s[n..])
}

fn get_priority(c: char) -> i64 {
    let n = c as i64;
    if n >= 'a' as i64 && n <= 'z' as i64 {
        n - ('a' as i64) + 1
    } else if n >= 'A' as i64 && n <= 'Z' as i64 {
        n - ('A' as i64) + 27
    } else {
        0
    }
}

fn get_matching(a: &str, b: &str) -> String {
    let ach: HashSet<char> = a.chars().collect();
    let bch: HashSet<char> = b.chars().collect();
    ach.intersection(&bch).collect()
}

fn get_matching3(a: &str, b: &str, c: &str) -> String {
    get_matching(a, &get_matching(b, c))
}


fn problem03() {    
    let data = read_lines(&data_file(3)).unwrap();
    let halves: Vec<(&str, &str)> = data.iter().map(|s| halve(s)).collect();
    let res1: i64 = halves.iter()
        .map(|(a, b)| get_matching(a, b).chars().map(|c| get_priority(c)).sum::<i64>())
        .sum();
    let res2: i64 = data
        .chunks(3)
        .map(|s| get_matching3(&s[0], &s[1], &s[2]).chars().map(|c| get_priority(c)).sum::<i64>())
        .sum();
    println!("Answer 1: {:?}, Answer 2: {:?}", res1, res2);
}

problem03();

Answer 1: 8085, Answer 2: 2515


In [166]:
// DAY04
#[macro_use] extern crate lazy_static;
extern crate regex;
use regex::Regex;

fn parse(s: &str) -> Vec<i64> {
    lazy_static! {
        static ref regex: Regex = Regex::new(r"-|,").unwrap();
    }
    regex.split(s)
        .map(|val| val.parse::<i64>().unwrap())
        .collect()
}

fn contains(x: i64, y: i64, a: i64, b: i64) -> bool {
    x >= a && y <= b
}

fn overlaps(x: i64, y: i64, a: i64, b: i64) -> bool {
    (x <= b && y >= a) || (a <= y && b >= x)
}
    
fn problem04() {    
    let data: Vec<Vec<i64>> = read_lines(&data_file(4))
        .unwrap()
        .iter()
        .map(|s| parse(s))
        .collect();
    
    let res1: i64 = data.iter()
        .map(|s| (contains(s[0], s[1], s[2], s[3]) || contains(s[2], s[3], s[0], s[1])) as i64)
        .sum();
    let res2: i64 = data.iter()
        .map(|s| overlaps(s[0], s[1], s[2], s[3]) as i64)
        .sum();
    println!("Answer 1: {}, Answer 2: {}", res1, res2);
}

problem04();

Answer 1: 483, Answer 2: 874


In [64]:
// DAY05
#[derive(Debug, PartialEq, Copy, Clone)]
struct CratesMove {
    src: usize,
    dst: usize,
    num: usize,
}

impl FromStr for CratesMove {
    type Err = std::num::ParseIntError;
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let parts: Vec<&str> = s.split(" ").collect();
        Ok(CratesMove {
            src: parts[3].parse()?,
            dst: parts[5].parse()?,
            num: parts[1].parse()?,
        })
    }
}

#[derive(Debug, PartialEq, Clone)]
struct CratesField {
    crates: Vec<Vec<char>>,
    tops: Vec<usize>,
}

impl CratesField {
    fn apply_move(&mut self, mv: &CratesMove, reverse: bool) {
        let src = mv.src - 1;
        let dst = mv.dst - 1;
        let w = self.crates[0].len();
        let ts = self.tops[src];
        let td = self.tops[dst];
        for i in 0..mv.num {
            let ps = ts - i - 1;
            let pd = if reverse {td + i} else {td + mv.num - i - 1};
            while self.crates.len() <= pd {
                self.crates.push(vec![' '; w]);
            }
            self.crates[pd][dst] = self.crates[ps][src];
            self.crates[ps][src] = ' ';
        }
        self.tops[src] -= mv.num;
        self.tops[dst] += mv.num;
    }
    
    fn get_top(self) -> String {
        self.tops.iter().enumerate().map(|(i, t)| self.crates[t - 1][i]).collect()
    }
}

fn parse_crates(lines: &Vec<&str>) -> CratesField {
    let h = lines.len() - 1;
    let w = (lines[0].len() + 1) / 4;
    let mut crates = vec![vec![' '; w]; h];
    let mut tops = vec![0; w];
    for j in 0..h {
        let line: Vec<_> = lines[j].chars().collect();
        for i in 0..w {
            let c = line[1 + i * 4];
            crates[j][i] = c;
            if c == ' ' {
                tops[i] += 1;
            }
        }
    }
    crates.reverse();
    for i in 0..w {
        tops[i] = h - tops[i];
    }
    CratesField {
        crates: crates,
        tops: tops,
    }
}

fn problem05() {  
    let data = read_string(&data_file(5)).unwrap();
    let parts: Vec<Vec<&str>> = data.split("\n\n").map(|s| s.split("\n").collect()).collect();
    let moves: Vec<CratesMove> = parts[1].iter().map(|s| s.parse().unwrap()).collect();
    
    let mut field1 = parse_crates(&parts[0]);
    let mut field2 = field1.clone();
    for m in &moves {
        &field1.apply_move(&m, true);
    }
    let res1 = &field1.get_top();

    for m in &moves {
        field2.apply_move(&m, false);
    }
    let res2 = &field2.get_top();
    println!("Answer 1: {}, Answer 2: {}", res1, res2);
}

problem05();

Answer 1: ZWHVFWQWW, Answer 2: HZFZCCWWV
